In [ ]:
import json
import csv
import pandas as pd

In [ ]:
# reading results
with open("odwn_query_results.json","r") as jf:
    odwn_results = json.load(jf)

In [ ]:
# importing query terms
with open("/LODlit/query_terms.json","r") as jf:
    query_terms = json.load(jf)

### 1. N synsets and LE (without synsets) by query term

In [ ]:
with open("odwn_synsets_LE_by_query_term.csv","w") as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","query_term","n_synsets", "n_LE"]
    writer.writerow(header)
    
    for query_term, results in odwn_results.items():
        # taking only Dutch terms
        for l, wordforms in query_terms["nl"].items():
            if query_term in wordforms:
                lemma = l
                
            n_synsets = len(set([hit["synset_id"] for hit in results if hit["synset_id"] != ""]))
            n_LE = len(set([hit["le_id"] for hit in results if hit["synset_id"] == ""]))
            
        row = [lemma,query_term,n_synsets,n_LE]
        writer.writerow(row)

### 2. N synsets and LE (without synsets) by lemma

In [ ]:
df = pd.read_csv("odwn_synsets_LE_by_query_term.csv")

In [ ]:
with open("odwn_synsets_LE_by_lemma.csv","w") as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","n_synsets","n_LE"]
    writer.writerow(header)

    for group in df.groupby("lemma"):
        row = [group[0],sum(group[1]["n_synsets"]),sum(group[1]["n_LE"])]
        writer.writerow(row)

### 3. N hits (occurences) by query term

In [ ]:
with open("odwn_hits_by_query_term.csv","w") as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","query_term","le_written_form","sense_definition","sense_examples","synset_definition","total_query_term"]
    writer.writerow(header)
    
    for query_term, results in odwn_results.items():
        
        le_written_form = 0
        sense_definition = 0
        sense_examples = 0
        synset_definition = 0
        
        # taking only Dutch terms
        for l, wordforms in query_terms["nl"].items():
            if query_term in wordforms:
                lemma = l
        
        # checking where query terms were found
        for hit in results:
            if hit["found_in"] == "le":
                le_written_form += 1
            if hit["found_in"] == "sense_definition":
                sense_definition += 1
            if hit["found_in"] == "sense_examples":
                sense_examples += 1
            if hit["found_in"] == "synset_definitions":
                synset_definition += 1
                
        total_count = le_written_form + sense_definition + sense_examples + synset_definition
        row = [lemma,query_term,le_written_form,sense_definition,sense_examples,synset_definition,total_count]
        writer.writerow(row)

### 4. N hits (occurences) by lemma

In [ ]:
df = pd.read_csv("odwn_hits_by_query_term.csv")

In [ ]:
with open("odwn_hits_by_lemma.csv","w") as csv_file:
    writer = csv.writer(csv_file)
    header = ["lemma","le_written_form","sense_definition","sense_examples","synset_definition","total_lemma"]
    writer.writerow(header)

    for group in df.groupby("lemma"):
        row = [group[0],sum(group[1]["le_written_form"]),sum(group[1]["sense_definition"]),\
              sum(group[1]["sense_examples"]),sum(group[1]["synset_definition"]),sum(group[1]["total_per_query_term"])]
        writer.writerow(row)